In [19]:
import os
from dotenv import load_dotenv
from langchain import OpenAI
from langchain.chains import ConversationalRetrievalChain
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain_core.document_loaders import BaseLoader
from langchain_core.documents import Document

In [11]:
load_dotenv()

True

In [16]:
llm = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
embeddings = OpenAIEmbeddings(api_key=os.getenv("OPENAI_API_KEY"))

In [13]:

# Load documents
documents = [
    {"title": "Document 1", "content": "This is the content of document 1."},
    {"title": "Document 2", "content": "This is the content of document 2."},
    # Add more documents as needed
]


In [21]:

# Create a document loader
class MyDocumentLoader(BaseLoader):
    def load(self):
        return [Document(page_content=doc['content']) for doc in documents]

document_loader = MyDocumentLoader()

# Load the documents
docs = document_loader.load()

In [22]:
# Create a vector store with FAISS
vectorstore = FAISS.from_documents(docs, embeddings)

# Initialize the retriever
retriever = vectorstore.as_retriever()

ConnectionError: HTTPSConnectionPool(host='openaipublic.blob.core.windows.net', port=443): Max retries exceeded with url: /encodings/cl100k_base.tiktoken (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x0000012AD3F25060>: Failed to resolve 'openaipublic.blob.core.windows.net' ([Errno 11001] getaddrinfo failed)"))

In [15]:
# Create the RAG pipeline
rag_chain = ConversationalRetrievalChain(
    retriever=retriever,
    llm=llm,
    return_source_documents=True
)

AttributeError: 'dict' object has no attribute 'page_content'

In [ ]:
# Example query
query = "Tell me about document 1."

# Run the RAG pipeline
response = rag_chain({"question": query})
print("Response:", response["text"])
print("Source Documents:", response["source_documents"])